<a href="https://colab.research.google.com/github/elenabcc/openai_textgeneration/blob/main/openai_assistants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai==1.25.0

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key="<your_api_key>"
)

# Create an Assistant

**NOTE**: if you created the assistant in the Playground you can skip the following cells.

## Upload tool resources

Upload files that can be used by the assistant. Needed both for the `code_interpreter` and the `file_search` tools.

Files are uploaded using the `File` upload endpoint and must have the purpose set to `assistants` to be used with this API.

In [ ]:
ek_file = client.files.create(
  file=open("exploding_kittens.pdf", "rb"),
  purpose='assistants'
)

ttr_file = client.files.create(
  file=open("ticket_to_ride.pdf", "rb"),
  purpose='assistants'
)

## Create the Assistant

To get started, creating an Assistant only requires specifying the `model` to use. But you can further customize the behavior of the Assistant:

- Use the `instructions` parameter to guide the personality of the Assistant and define its goals. Instructions are similar to system messages in the Chat Completions API.
- Use the `tools` parameter to give the Assistant access to up to 128 tools. You can give it access to OpenAI-hosted tools like `code_interpreter` and `file_search`, or call a third-party tools via a `function` calling.
- Use the `tool_resources` parameter to give the tools like `code_interpreter` and `file_search` access to files.

In [ ]:
assistant = client.beta.assistants.create(
  name="Board Game Buddy",
  instructions="You are a board game expert. Your task is to explain to a six-year-old how to play Exploding Kittens and Ticket to Ride. Please use a simple language so that the kid can understand the rules.",
  tools=[{"type": "file_search"}],
  model="gpt-4-turbo",
  tool_resources={
    "file_search": {
      "file_ids": [ek_file.id, ttr_file.id]
    }
  }
)

# Load an Assistant

If you want to use an assistant you already created run this cell.

In [ ]:
assistant = client.beta.assistants.retrieve(
    assistant_id="<your_assistant_id>"
)
assistant.dict()


# Create a Thread
A Thread represents a conversation between a user and one or many Assistants. You can create a Thread when a user (or your AI application) starts a conversation with your Assistant.

In [ ]:
thread = client.beta.threads.create()

# Add a Message to the Thread
The contents of the messages your users or applications create are added as Message objects to the Thread. Messages can contain both text and files. There is no limit to the number of Messages you can add to Threads — we smartly truncate any context that does not fit into the model's context window.


In [ ]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="come si gioca a exploding kittens?"
)

# Create a Run
Once all the user Messages have been added to the Thread, you can Run the Thread with any Assistant. Creating a Run uses the model and tools associated with the Assistant to generate a response. These responses are added to the Thread as assistant Messages.

Runs are asynchronous, which means you'll want to monitor their status by polling the Run object until a terminal status is reached. For convenience, the 'create and poll' SDK helpers assist both in creating the run and then polling for its completion.

In [ ]:

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)


Once the Run completes, you can list the Messages added to the Thread by the Assistant.

In [ ]:

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
else:
  print(run.status)

## Override configurations

By default, a Run will use the model and tools configuration specified in Assistant object, but you can override most of these when creating the Run for added flexibility.

**NOTE**: The only configuration that cannot be overridden during Run creation is `tool_resources`, you must use the modify Assistant endpoint to do that.

In [ ]:

run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="You are an extremely ironic board game expert. Your task is to explain how to play Exploding Kittens and Ticket to Ride. Please use irony and sarcasm in your answers.",
)

In [ ]:

if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages.data[0].content[0].text.value)
else:
  print(run.status)